In [1]:
dataframe = spark.sql("SELECT * FROM bing_api_db.tbl_api_news LIMIT 1000")
display(dataframe)

StatementMeta(, 55f0516b-98cf-4f62-bd48-837ff33e205a, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d7e380cb-01ea-4755-92fd-96f77e4591b5)

In [2]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText
from pyspark.sql.functions import col
from pyspark.sql.functions import col, to_date
from pyspark.sql.utils import AnalysisException

StatementMeta(, 55f0516b-98cf-4f62-bd48-837ff33e205a, 4, Finished, Available, Finished)

In [3]:
mdl = (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))
        

StatementMeta(, 55f0516b-98cf-4f62-bd48-837ff33e205a, 5, Finished, Available, Finished)

In [4]:
outcome = mdl.transform(dataframe)

StatementMeta(, 55f0516b-98cf-4f62-bd48-837ff33e205a, 6, Finished, Available, Finished)

In [5]:
display(outcome)

StatementMeta(, 55f0516b-98cf-4f62-bd48-837ff33e205a, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 66e1586b-9b1c-40b5-8061-08cd3a0d2f38)

In [7]:
sentimentdataframe = outcome.withColumn("sentiment", col("response.documents.sentiment"))

StatementMeta(, 55f0516b-98cf-4f62-bd48-837ff33e205a, 9, Finished, Available, Finished)

In [8]:
display(sentimentdataframe)

StatementMeta(, 55f0516b-98cf-4f62-bd48-837ff33e205a, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 995347df-55f1-4e9c-a670-45c310c57025)

In [9]:
finalsentiment = sentimentdataframe.drop("error","response")

StatementMeta(, 55f0516b-98cf-4f62-bd48-837ff33e205a, 11, Finished, Available, Finished)

In [10]:
display(finalsentiment)

StatementMeta(, 55f0516b-98cf-4f62-bd48-837ff33e205a, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 79b2c58a-4772-4dfe-8d76-523f34417552)

In [11]:
finalsentiment = finalsentiment.withColumn("datePublished", to_date(col("datePublished"),"dd-MMM-yyyy"))

StatementMeta(, 55f0516b-98cf-4f62-bd48-837ff33e205a, 13, Finished, Available, Finished)

In [13]:
try:
    tabletag = 'bing_api_db.tbl_API_sentiment'
    finalsentiment.write.format("delta").saveAsTable(tabletag)

except AnalysisException:

    print(" Table already available")

    finalsentiment.createOrReplaceTempView("vw_finaldataframesentiment")

    spark.sql(f"""  MERGE INTO {tabletag} aimedtable
                    USING vw_finaldataframesentiment originview
                    ON originview.url = aimedtable.url
                    WHEN MATCHED AND
                    originview.title <> aimedtable.title OR 
                    originview.description <> aimedtable.description OR
                    originview.category <> aimedtable.category OR 
                    originview.image <> aimedtable.image OR
                    originview.provider <> aimedtable.provider OR 
                    originview.datePublished <> aimedtable.datePublished 

                    THEN UPDATE SET *

                    WHEN NOT MATCHED THEN INSERT *
                    
                """)

StatementMeta(, 55f0516b-98cf-4f62-bd48-837ff33e205a, 15, Finished, Available, Finished)

 Table already available
